In [2]:
!pip install "numpy<2.0" "torch==2.2.0" pandas numpy==1.26.4 scikit-learn tqdm --force-reinstall

  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached torch-2.2.0-cp312-cp312-manylinux1_x86_64.whl.metadata (25 kB)
  Using cached pandas-2.3.3-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (91 kB)
  Using cached scikit_learn-1.7.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.6-py3-none-any.whl.metadata (6.8 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.10.0-py3-none-any.whl.metadata (10 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
Traceback (most recent call last):
  File "/usr/loc

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# ============================================
# Step 1: 下载并加载 MovieLens-1M
# ============================================
import zipfile
import urllib.request
from collections import defaultdict
import os, math, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from collections import Counter

try:
    PROJECT_ROOT
except NameError:
    PROJECT_ROOT = Path('/content/drive/MyDrive')

DATA_DIR = PROJECT_ROOT / 'ml-25m'
FIG_DIR = PROJECT_ROOT / 'figs'
FIG_DIR.mkdir(parents=True, exist_ok=True)



In [5]:
def load_if_needed():
    """Load ratings/movies if not already in memory."""
    glb = globals()
    if 'ratings' in glb and 'movies' in glb:
        return glb['ratings'], glb['movies']
    r = pd.read_csv(DATA_DIR/'ratings.csv',
                    usecols=['userId','movieId','rating','timestamp'],
                    dtype={'userId':'int32','movieId':'int32','rating':'float32','timestamp':'int64'},
                    engine='pyarrow')
    m = pd.read_csv(DATA_DIR/'movies.csv',
                    usecols=['movieId','title','genres'],
                    dtype={'movieId':'int32','title':'string','genres':'string'},
                    engine='pyarrow')
    return r, m

ratings, movies = load_if_needed()

print("Ratings shape:", ratings.shape)
print("Movies shape:", movies.shape)



Ratings shape: (25000095, 4)
Movies shape: (62423, 3)


In [6]:
# ============================================
# Step 2: 基本预处理
#  - 只保留评分 >= 4 作为“正反馈”
#  - 只保留交互数 >= 20 的用户，方便做 time split
# ============================================
# 只用隐式反馈：rating >= 4
ratings_imp = ratings[ratings["rating"] >= 4].copy()

# 过滤交互太少的用户
user_counts = ratings_imp["userId"].value_counts()
valid_users = user_counts[user_counts >= 20].index
ratings_imp = ratings_imp[ratings_imp["userId"].isin(valid_users)]

# 重新映射 user_id, movie_id
unique_users = ratings_imp["userId"].unique()
unique_items = ratings_imp["movieId"].unique()

user2idx = {u: i for i, u in enumerate(sorted(unique_users))}
item2idx = {m: i for i, m in enumerate(sorted(unique_items))}

idx2user = {i: u for u, i in user2idx.items()}
idx2item = {i: m for m, i in item2idx.items()}

ratings_imp["u"] = ratings_imp["userId"].map(user2idx)
ratings_imp["i"] = ratings_imp["movieId"].map(item2idx)

num_users = len(user2idx)
num_items = len(item2idx)
print("num_users:", num_users, "num_items:", num_items)

# 只保留在映射里的电影元数据
movies = movies[movies["movieId"].isin(unique_items)].copy()
movies["item_idx"] = movies["movieId"].map(item2idx)



num_users: 124514 num_items: 40736


In [7]:
# ============================================
# Step 3: 构造 Genre 兴趣空间
#  - 每部电影有一个多标签 genres 字段，用它构造 multi-hot 向量
#  - 对每个用户：历史兴趣分布 h_u (按 genre 归一化)
# ============================================
# 解析 genres
movies["genres_list"] = movies["genres"].apply(lambda x: x.split("|") if isinstance(x, str) else [])

all_genres = sorted({g for gl in movies["genres_list"] for g in gl if g != "(no genres listed)"})
genre2idx = {g: idx for idx, g in enumerate(all_genres)}
num_genres = len(all_genres)
print("Number of genres:", num_genres)
print(all_genres)

# item -> genre multi-hot
item_genre = np.zeros((num_items, num_genres), dtype=np.float32)
for _, row in movies.iterrows():
    i = row["item_idx"]
    for g in row["genres_list"]:
        if g in genre2idx:
            item_genre[i, genre2idx[g]] = 1.0

# 防止全零
item_genre[item_genre.sum(axis=1) == 0] = 1.0 / num_genres

# 归一化为概率分布（每个 item 的 genre 分布）
item_genre = item_genre / item_genre.sum(axis=1, keepdims=True)

# 全局 genre 分布 g (后面 IPE 用)
global_genre = item_genre.mean(axis=0)
global_genre = global_genre / global_genre.sum()

print("Global genre distribution:", global_genre)



Number of genres: 19
['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
Global genre distribution: [0.04955154 0.02814571 0.03056484 0.02408399 0.17234246 0.03620703
 0.09420056 0.27095878 0.020441   0.00542865 0.05194057 0.00411767
 0.01160705 0.0198238  0.05897246 0.02699426 0.06491237 0.01680104
 0.01290618]


In [8]:
# ============================================
# Step 4: 按时间对每个用户做 60/20/20 划分
#  - past: 用于训练 + 历史兴趣 h_u
#  - mid: 作为 echo 形成的中期阶段，用于做分析
#  - future: 用于评估 NDCG/Recall
# ============================================
ratings_imp = ratings_imp.sort_values(["u", "timestamp"])

user_past = defaultdict(list)
user_mid = defaultdict(list)
user_future = defaultdict(list)

for u, group in ratings_imp.groupby("u"):
    items = group["i"].values
    ts = group["timestamp"].values
    n = len(items)
    if n < 5:
        continue
    p = int(n * 0.6)
    m = int(n * 0.8)
    past_indices = np.arange(0, p)
    mid_indices = np.arange(p, m)
    fut_indices = np.arange(m, n)
    for idx in past_indices:
        user_past[u].append(items[idx])
    for idx in mid_indices:
        user_mid[u].append(items[idx])
    for idx in fut_indices:
        user_future[u].append(items[idx])

# 只保留有 future 的用户
valid_users2 = [u for u in user_future.keys() if len(user_past[u]) > 0]
print("Users with past+future:", len(valid_users2))

# 构造训练正样本列表
train_pairs = []
for u in valid_users2:
    for i in user_past[u]:
        train_pairs.append((u, i))

len(train_pairs)



Users with past+future: 124514


7127598

In [9]:
# ============================================
# Step 5: 构造用户历史兴趣分布 h_u（基于 past）
# ============================================
user_hist_genre = np.zeros((num_users, num_genres), dtype=np.float32)

for u in valid_users2:
    items = user_past[u]
    if len(items) == 0:
        continue
    g_sum = item_genre[items].sum(axis=0)
    if g_sum.sum() == 0:
        g_sum = np.ones(num_genres) / num_genres
    user_hist_genre[u] = g_sum / g_sum.sum()

# 对于没有 past 的，设为全局分布
for u in range(num_users):
    if user_hist_genre[u].sum() == 0:
        user_hist_genre[u] = global_genre



In [10]:
# ============================================
# Step 6: 定义 BPR-MF 模型 (PyTorch)
# ============================================
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

class BPRMF(nn.Module):
    def __init__(self, num_users, num_items, embed_dim=32):
        super().__init__()
        self.user_emb = nn.Embedding(num_users, embed_dim)
        self.item_emb = nn.Embedding(num_items, embed_dim)
        nn.init.normal_(self.user_emb.weight, std=0.01)
        nn.init.normal_(self.item_emb.weight, std=0.01)

    def forward(self, user_ids, item_ids):
        u = self.user_emb(user_ids)
        i = self.item_emb(item_ids)
        return (u * i).sum(dim=-1)

    def full_rank(self, user_ids):
        """对一批用户返回所有 item 的得分矩阵 (batch_size, num_items)"""
        u = self.user_emb(user_ids)  # (B, D)
        i = self.item_emb.weight     # (I, D)
        scores = torch.matmul(u, i.t())
        return scores

class BPRDataset(Dataset):
    def __init__(self, train_pairs, num_items, user_past_dict, num_neg=1):
        self.train_pairs = train_pairs
        self.num_items = num_items
        self.user_past_dict = user_past_dict
        self.num_neg = num_neg

    def __len__(self):
        return len(self.train_pairs)

    def __getitem__(self, idx):
        u, i_pos = self.train_pairs[idx]
        # 随机负采样（不在 past 中）
        pos_set = set(self.user_past_dict[u])
        i_neg = np.random.randint(0, self.num_items)
        tries = 0
        while i_neg in pos_set and tries < 10:
            i_neg = np.random.randint(0, self.num_items)
            tries += 1
        return u, i_pos, i_neg



Using device: cuda


In [11]:
# ============================================
# Step 7: Baseline BPR 训练函数
# ============================================
def train_bpr(model, train_pairs, user_past_dict, num_items,
              epochs=10, batch_size=2048, lr=1e-3, l2=1e-6):
    dataset = BPRDataset(train_pairs, num_items, user_past_dict)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0.0)

    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        for u, i_pos, i_neg in tqdm(loader, desc=f"BPR Epoch {epoch+1}/{epochs}", leave=False):
            u = u.to(device)
            i_pos = i_pos.to(device)
            i_neg = i_neg.to(device)

            pos_scores = model(u, i_pos)
            neg_scores = model(u, i_neg)
            loss_bpr = -torch.log(torch.sigmoid(pos_scores - neg_scores) + 1e-8).mean()

            # L2 正则
            reg = (model.user_emb(u).pow(2).sum(dim=1) +
                   model.item_emb(i_pos).pow(2).sum(dim=1) +
                   model.item_emb(i_neg).pow(2).sum(dim=1)).mean()
            loss = loss_bpr + l2 * reg

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        print(f"[BPR] Epoch {epoch+1} loss = {total_loss / len(loader):.4f}")

    return model



In [16]:
# ============================================
# 修正版 IPE 训练函数：每个 user 精确采样 C 个候选
# ============================================
def train_bpr_ipe(model, train_pairs, user_past_dict, num_items,
                  user_hist_genre, global_genre,
                  epochs=10, batch_size=2048, lr=1e-3,
                  l2=1e-6, gamma=0.05, beta=0.3,
                  num_candidates_per_user=40, temperature=1.0):
    """
    注意：
    - 对 batch 里的每个用户，每次循环严格采样 num_candidates_per_user 个候选 item
    - 确保 cand_list 拼接后长度 = len(batch_users) * num_candidates_per_user
    - 然后才能安全地 scores.view(len(batch_users), num_candidates_per_user)
    """
    dataset = BPRDataset(train_pairs, num_items, user_past_dict)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0.0)

    user_hist_genre_t = torch.tensor(user_hist_genre, dtype=torch.float32, device=device)
    global_genre_t = torch.tensor(global_genre, dtype=torch.float32, device=device)
    item_genre_t = torch.tensor(item_genre, dtype=torch.float32, device=device)

    for epoch in range(epochs):
        model.train()
        total_loss = 0.0

        for u, i_pos, i_neg in tqdm(loader, desc=f"IPE Epoch {epoch+1}/{epochs}", leave=False):
            u = u.to(device)
            i_pos = i_pos.to(device)
            i_neg = i_neg.to(device)

            # --------- BPR 主损失 ----------
            pos_scores = model(u, i_pos)
            neg_scores = model(u, i_neg)
            loss_bpr = -torch.log(torch.sigmoid(pos_scores - neg_scores) + 1e-8).mean()

            reg = (
                model.user_emb(u).pow(2).sum(dim=1) +
                model.item_emb(i_pos).pow(2).sum(dim=1) +
                model.item_emb(i_neg).pow(2).sum(dim=1)
            ).mean()

            # --------- Preference Expansion ----------
            batch_users = u.detach().cpu().numpy()  # 长度 = 当前 batch size
            C = num_candidates_per_user

            cand_items_list = []
            cand_users_list = []

            for uu in batch_users:
                pos_list = list(set(user_past_dict[int(uu)]))  # 该 user 的所有正样本
                cand = []

                # 如果历史看过的电影 >= C，就从中不放回采样 C 个
                if len(pos_list) >= C:
                    cand = list(
                        np.random.choice(pos_list, size=C, replace=False)
                    )
                else:
                    # 否则：先把所有 past 塞进去，再随机补到 C 个
                    cand = pos_list[:]
                    cand_set = set(cand)
                    while len(cand) < C:
                        rid = np.random.randint(0, num_items)
                        if rid not in cand_set:
                            cand.append(rid)
                            cand_set.add(rid)

                # 此时保证 len(cand) == C
                assert len(cand) == C
                cand_items_list.extend(cand)
                cand_users_list.extend([uu] * C)

            cand_items = torch.tensor(cand_items_list, dtype=torch.long, device=device)    # (B*C,)
            cand_users = torch.tensor(cand_users_list, dtype=torch.long, device=device)    # (B*C,)

            # 防御性检查（可选）
            B = len(batch_users)
            assert cand_items.shape[0] == B * C, f"cand_items len={cand_items.shape[0]}, expected {B*C}"

            scores = model(cand_users, cand_items)  # (B*C,)
            scores = scores.view(B, C)              # (B, C)

            # softmax 得到推荐权重
            weights = F.softmax(scores / temperature, dim=1)  # (B, C)

            cand_items_mat = cand_items.view(B, C)            # (B, C)
            cand_genre = item_genre_t[cand_items_mat]         # (B, C, G)

            # r_u: 推荐 genre 分布 (B, G)
            r_u = (weights.unsqueeze(-1) * cand_genre).sum(dim=1)
            r_u = r_u / (r_u.sum(dim=1, keepdim=True) + 1e-8)

            # 历史 & 目标分布
            h_u_batch = user_hist_genre_t[u]                  # (B, G)
            t_u = (1 - beta) * h_u_batch + beta * global_genre_t
            t_u = t_u / (t_u.sum(dim=1, keepdim=True) + 1e-8)

            # KL(r_u || t_u)
            kl = (r_u * (torch.log(r_u + 1e-8) - torch.log(t_u + 1e-8))).sum(dim=1)
            loss_pe = kl.mean()

            loss = loss_bpr + l2 * reg + gamma * loss_pe

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"[IPE] Epoch {epoch+1} loss = {total_loss / len(loader):.4f}")

    return model


In [22]:
# ============================================
# Step 9: 评估函数
#  - NDCG@K & Recall@K (future 部分)
#  - 长尾曝光比例
#  - Intra-list Similarity (基于 genre)
#  - Echo Chamber 指标 (ECS)：
#      ECS1 = max(0, H(h_u) - H(r_u))
#      ECS2 = KL(r_u || hist)   (贴脸程度)
# ============================================
from math import log2

def ndcg_at_k(recommended, ground_truth, k=10):
    if not ground_truth:
        return 0.0
    dcg = 0.0
    for rank, i in enumerate(recommended[:k]):
        if i in ground_truth:
            dcg += 1.0 / log2(rank + 2)
    # ideal DCG: min(len(gt), k) relevant items at top
    ideal_hits = min(len(ground_truth), k)
    idcg = sum(1.0 / log2(rank + 2) for rank in range(ideal_hits))
    return dcg / idcg if idcg > 0 else 0.0

def recall_at_k(recommended, ground_truth, k=10):
    if not ground_truth:
        return 0.0
    hits = len(set(recommended[:k]) & set(ground_truth))
    return hits / len(ground_truth)

# 计算每个 user 推荐 topK
def get_topk_for_all_users(model, K=50, exclude_train=True):
    """
    为所有 user 生成推荐列表：
    - 只屏蔽训练集中的 past（可以选择是否加 mid）
    - 不要把 future 也屏蔽掉，否则永远 hit 不了
    """
    model.eval()
    all_topk = {}
    with torch.no_grad():
        batch_size = 512
        for start in range(0, num_users, batch_size):
            end = min(num_users, start + batch_size)
            users_batch = list(range(start, end))
            users_t = torch.tensor(users_batch, dtype=torch.long, device=device)
            scores = model.full_rank(users_t)  # (B, num_items)

            if exclude_train:
                for idx, u in enumerate(users_batch):
                    # ✅ 只屏蔽 past（训练过的），mid/future 不要动
                    seen = set(user_past[u])   # + user_mid[u] 如果你希望也不推荐 mid
                    if len(seen) > 0:
                        scores[idx, list(seen)] = -1e9

            topk_scores, topk_items = torch.topk(scores, K, dim=1)
            for idx, u in enumerate(users_batch):
                all_topk[u] = topk_items[idx].cpu().numpy().tolist()
    return all_topk


# 计算 genre 分布 & ECS 指标
def compute_genre_metrics(topk_dict, K_eval=10):
    # popularity for long-tail (基于 training past)
    item_pop = np.zeros(num_items, dtype=np.int32)
    for u in valid_users2:
        for i in user_past[u]:
            item_pop[i] += 1
    # 定义长尾：popularity 排序后，后 50% 作为 tail
    sorted_idx = np.argsort(item_pop)
    threshold_index = int(len(sorted_idx) * 0.5)
    tail_items = set(sorted_idx[:threshold_index])

    ndcgs = []
    recalls = []
    ils_list = []
    tail_ratios = []
    ecs1_list = []  # H(hist) - H(rec)
    ecs2_list = []  # KL(r_u || hist)

    eps = 1e-8

    for u in valid_users2:
        rec_items = topk_dict[u][:K_eval]
        if len(rec_items) == 0:
            continue
        gt = user_future[u]

        # Accuracy
        ndcgs.append(ndcg_at_k(rec_items, gt, k=K_eval))
        recalls.append(recall_at_k(rec_items, gt, k=K_eval))

        # Intra-list similarity (基于 genre cosine)
        # 取 item_genre 中对应行
        g_mat = item_genre[rec_items]  # (K, G)
        # 余弦相似度矩阵
        norm = np.linalg.norm(g_mat, axis=1, keepdims=True) + eps
        g_norm = g_mat / norm
        sim_mat = np.dot(g_norm, g_norm.T)
        # 只算上三角不含对角线
        k = len(rec_items)
        if k > 1:
            ils = (sim_mat.sum() - np.trace(sim_mat)) / (k * (k - 1))
        else:
            ils = 0.0
        ils_list.append(ils)

        # Tail ratio
        tail_count = sum(1 for i in rec_items if i in tail_items)
        tail_ratios.append(tail_count / len(rec_items))

        # Genre 分布 r_u
        g_sum = g_mat.sum(axis=0)
        r_u = g_sum / (g_sum.sum() + eps)

        # 历史分布 h_u
        h_u = user_hist_genre[u]
        # 熵
        def entropy(p):
            p2 = p[p > 0]
            return -np.sum(p2 * np.log(p2))
        H_hist = entropy(h_u)
        H_rec = entropy(r_u)
        ecs1 = max(0.0, H_hist - H_rec)  # 如果推荐更窄，则 >0
        ecs1_list.append(ecs1)

        # KL(r_u || h_u)
        p = r_u
        q = h_u + eps
        ecs2 = np.sum(p * (np.log(p + eps) - np.log(q)))
        ecs2_list.append(ecs2)

    results = {
        "NDCG@{}".format(K_eval): float(np.mean(ndcgs)) if ndcgs else 0.0,
        "Recall@{}".format(K_eval): float(np.mean(recalls)) if recalls else 0.0,
        "ILS@{}".format(K_eval): float(np.mean(ils_list)) if ils_list else 0.0,
        "TailRatio@{}".format(K_eval): float(np.mean(tail_ratios)) if tail_ratios else 0.0,
        "ECS1_hist_minus_rec_entropy": float(np.mean(ecs1_list)) if ecs1_list else 0.0,
        "ECS2_KL_rec_hist": float(np.mean(ecs2_list)) if ecs2_list else 0.0,
    }
    return results



In [23]:
# ============================================
# Step 10: 训练 Baseline BPR
# ============================================
embed_dim = 32
epochs_bpr = 8
epochs_ipe = 8  # 可以根据时间调整

bpr_model = BPRMF(num_users, num_items, embed_dim=embed_dim)
bpr_model = train_bpr(
    bpr_model,
    train_pairs,
    user_past,
    num_items,
    epochs=epochs_bpr,
    batch_size=2048,
    lr=1e-3,
    l2=1e-6
)

# 得到推荐列表 & 评估
topk_bpr = get_topk_for_all_users(bpr_model, K=50)
metrics_bpr = compute_genre_metrics(topk_bpr, K_eval=10)
print("==== Baseline BPR Metrics ====")
for k, v in metrics_bpr.items():
    print(f"{k}: {v:.4f}")



BPR Epoch 1/8:   0%|          | 0/3480 [00:00<?, ?it/s]

[BPR] Epoch 1 loss = 0.1893


BPR Epoch 2/8:   0%|          | 0/3480 [00:00<?, ?it/s]

[BPR] Epoch 2 loss = 0.0635


BPR Epoch 3/8:   0%|          | 0/3480 [00:00<?, ?it/s]

...

[BPR] Epoch 6 loss = 0.0480


BPR Epoch 7/8:   0%|          | 0/3480 [00:00<?, ?it/s]

[BPR] Epoch 7 loss = 0.0449


BPR Epoch 8/8:   0%|          | 0/3480 [00:00<?, ?it/s]

[BPR] Epoch 8 loss = 0.0419
==== Baseline BPR Metrics ====
NDCG@10: 0.0645
Recall@10: 0.0414
ILS@10: 0.2915
TailRatio@10: 0.0000
ECS1_hist_minus_rec_entropy: 0.1417
ECS2_KL_rec_hist: 1.3075


In [24]:
# ============================================
# Step 11: 训练 IPE (BPR + Preference Expansion)
#  - gamma 控制“打破 echo chamber”的强度
#  - beta 控制 target 分布中全局成分占比
# ============================================
ipe_model = BPRMF(num_users, num_items, embed_dim=embed_dim)
ipe_model = train_bpr_ipe(
    ipe_model,
    train_pairs,
    user_past,
    num_items,
    user_hist_genre=user_hist_genre,
    global_genre=global_genre,
    epochs=epochs_ipe,
    batch_size=2048,
    lr=1e-3,
    l2=1e-6,
    gamma=0.1,      # 可以调大一点增强反 echo 效果
    beta=0.3,       # 0.0~0.5 之间试
    num_candidates_per_user=40,
    temperature=1.0
)

topk_ipe = get_topk_for_all_users(ipe_model, K=50)
metrics_ipe = compute_genre_metrics(topk_ipe, K_eval=10)
print("==== IPE (BPR + PE) Metrics ====")
for k, v in metrics_ipe.items():
    print(f"{k}: {v:.4f}")



IPE Epoch 1/8:   0%|          | 0/3480 [00:00<?, ?it/s]

[IPE] Epoch 1 loss = 0.2037


IPE Epoch 2/8:   0%|          | 0/3480 [00:00<?, ?it/s]

[IPE] Epoch 2 loss = 0.0763


IPE Epoch 3/8:   0%|          | 0/3480 [00:00<?, ?it/s]

...

[IPE] Epoch 6 loss = 0.0566


IPE Epoch 7/8:   0%|          | 0/3480 [00:00<?, ?it/s]

[IPE] Epoch 7 loss = 0.0532


IPE Epoch 8/8:   0%|          | 0/3480 [00:00<?, ?it/s]

[IPE] Epoch 8 loss = 0.0503
==== IPE (BPR + PE) Metrics ====
NDCG@10: 0.0454
Recall@10: 0.0288
ILS@10: 0.5822
TailRatio@10: 0.0000
ECS1_hist_minus_rec_entropy: 0.5536
ECS2_KL_rec_hist: 1.1165


In [25]:
# ============================================
# Step 12: 对比结果 & “证实问题”
#  - 看看：
#    1) NDCG/Recall 是否变化不大
#    2) ILS 是否下降（列表内部更不那么像）
#    3) TailRatio 是否上升（更多长尾）
#    4) ECS1 是否下降 （推荐不再比历史更窄那么多）
#    5) ECS2 是否下降 （推荐分布不再过度贴脸历史）
# ============================================
print("========== Final Comparison ==========")
print("Baseline BPR:")
for k, v in metrics_bpr.items():
    print(f"  {k}: {v:.4f}")

print("\nIPE (BPR + Preference Expansion):")
for k, v in metrics_ipe.items():
    print(f"  {k}: {v:.4f}")



========== Final Comparison ==========
Baseline BPR:
  NDCG@10: 0.0645
  Recall@10: 0.0414
  ILS@10: 0.2915
  TailRatio@10: 0.0000
  ECS1_hist_minus_rec_entropy: 0.1417
  ECS2_KL_rec_hist: 1.3075

IPE (BPR + Preference Expansion):
  NDCG@10: 0.0454
  Recall@10: 0.0288
  ILS@10: 0.5822
  TailRatio@10: 0.0000
  ECS1_hist_minus_rec_entropy: 0.5536
  ECS2_KL_rec_hist: 1.1165


In [26]:
# ============================================
# Cell A: 预计算流行度 / 长尾 / genre 矩阵
# ============================================
import numpy as np
import torch

eps = 1e-8

# 如果你之前已经算过 item_pop_global，可以直接复用
item_pop_global = np.zeros(num_items, dtype=np.int32)
for u in valid_users2:
    for i in user_past[u]:
        item_pop_global[i] += 1

# 定义长尾：底部 80% 作为 tail（可以自己调）
sorted_idx = np.argsort(item_pop_global)
threshold_index = int(len(sorted_idx) * 0.8)
tail_items_global = set(sorted_idx[:threshold_index])

print("Tail items ratio:", len(tail_items_global) / num_items)

# 转成 torch，方便后面用
item_genre_t = torch.tensor(item_genre, dtype=torch.float32, device=device)
global_genre_t = torch.tensor(global_genre, dtype=torch.float32, device=device)
user_hist_genre_t = torch.tensor(user_hist_genre, dtype=torch.float32, device=device)

item_pop_norm = item_pop_global.astype(np.float32)
if item_pop_norm.max() > 0:
    item_pop_norm = item_pop_norm / item_pop_norm.max()
item_pop_t = torch.tensor(item_pop_norm, dtype=torch.float32, device=device)


Tail items ratio: 0.799980361351139


In [27]:
# ============================================
# Cell B: 为每个 user 构造候选池 (top-L) + 分数
# ============================================
def get_candidate_pool_with_scores(model, L=100, exclude_train=True):
    model.eval()
    cand_items_dict = {}
    cand_scores_dict = {}

    with torch.no_grad():
        batch_size = 512
        for start in range(0, num_users, batch_size):
            end = min(num_users, start + batch_size)
            users_batch = list(range(start, end))
            users_t = torch.tensor(users_batch, dtype=torch.long, device=device)
            scores = model.full_rank(users_t)  # (B, num_items)

            if exclude_train:
                for idx, u in enumerate(users_batch):
                    seen = set(user_past[u])
                    if len(seen) > 0:
                        scores[idx, list(seen)] = -1e9

            # 取 top-L
            L_eff = min(L, num_items)
            topL_scores, topL_items = torch.topk(scores, L_eff, dim=1)

            for idx, u in enumerate(users_batch):
                cand_items_dict[u] = topL_items[idx].cpu().numpy()
                cand_scores_dict[u] = topL_scores[idx].cpu().numpy()

    return cand_items_dict, cand_scores_dict

L_candidates = 100  # 候选池大小，可以 100 / 200 自己试
cand_items_dict, cand_scores_dict = get_candidate_pool_with_scores(bpr_model, L=L_candidates)
print("Example candidate pool size:", len(cand_items_dict[valid_users2[0]]))


Example candidate pool size: 100


In [28]:
# ============================================
# Cell C: 单用户 Echo-Calibrated Re-ranking
# ============================================
def rerank_ecr_one_user(
    u,
    cand_items,
    cand_scores,
    K=10,
    alpha_global=0.3,    # 历史 vs 全局 混合：0.3 表示 30% global
    w_rel=0.5,           # 相关性权重
    w_cal=0.3,           # 校准（对齐 t_u）权重
    w_tail=0.2           # 长尾曝光权重
):
    """
    根据 Steck (2018) 的 calibration 思路 + echo-aware 修改，
    对用户 u 的候选列表做贪心重排，返回最终 top-K item 列表。

    cand_items: np.array, shape (L,)
    cand_scores: np.array, shape (L,)
    """
    L = len(cand_items)
    if L == 0:
        return []

    # 归一化相关性到 [0,1]
    scores = cand_scores.astype(np.float32)
    s_min, s_max = scores.min(), scores.max()
    if s_max - s_min < 1e-6:
        rel_norm = np.ones_like(scores)
    else:
        rel_norm = (scores - s_min) / (s_max - s_min)

    # tail 标记
    tail_flags = np.array(
        [1.0 if int(i) in tail_items_global else 0.0 for i in cand_items],
        dtype=np.float32
    )

    # 目标 genre 分布 t_u = (1-alpha)*hist + alpha*global
    hist = user_hist_genre[u].astype(np.float32)
    hist = hist / (hist.sum() + 1e-8)
    t_u = (1 - alpha_global) * hist + alpha_global * global_genre
    t_u = t_u / (t_u.sum() + 1e-8)

    chosen_indices = []        # 在 cand_items 中的索引
    remaining = list(range(L))

    # 当前已经选出的集合 S 的统计量
    g_sum = np.zeros_like(t_u, dtype=np.float32)  # genre 计数和
    sum_rel = 0.0
    tail_count = 0

    for step in range(K):
        if not remaining:
            break

        best_score = -1e9
        best_ridx = None

        for ridx in remaining:
            item_id = int(cand_items[ridx])

            # 新的 genre 统计
            g_i = item_genre[item_id]          # (G,)
            g_new = g_sum + g_i
            p_new = g_new / (g_new.sum() + 1e-8)

            # 校准项：-KL(p_new || t_u)
            kl = np.sum(p_new * (np.log(p_new + 1e-8) - np.log(t_u + 1e-8)))
            cal_score = -kl

            # 新的平均相关性
            rel_new = (sum_rel + rel_norm[ridx]) / (len(chosen_indices) + 1)

            # 新的 tail 比例
            tail_new = (tail_count + tail_flags[ridx]) / (len(chosen_indices) + 1)

            total = w_rel * rel_new + w_cal * cal_score + w_tail * tail_new

            if total > best_score:
                best_score = total
                best_ridx = ridx

        # 选中 best_ridx
        chosen_indices.append(best_ridx)
        remaining.remove(best_ridx)

        item_id = int(cand_items[best_ridx])
        g_sum += item_genre[item_id]
        sum_rel += rel_norm[best_ridx]
        tail_count += tail_flags[best_ridx]

    # 返回真正的 item id 列表
    reranked_items = [int(cand_items[idx]) for idx in chosen_indices]
    return reranked_items


In [29]:
# ============================================
# Cell D: 给所有用户做 ECR Re-ranking，并评估
# ============================================
def get_topk_ecr_for_all_users(
    cand_items_dict,
    cand_scores_dict,
    K=10,
    alpha_global=0.3,
    w_rel=0.5,
    w_cal=0.3,
    w_tail=0.2
):
    topk_ecr = {}
    for u in range(num_users):
        cand_items = cand_items_dict.get(u, None)
        cand_scores = cand_scores_dict.get(u, None)
        if cand_items is None or len(cand_items) == 0:
            topk_ecr[u] = []
            continue
        topk_ecr[u] = rerank_ecr_one_user(
            u,
            cand_items,
            cand_scores,
            K=K,
            alpha_global=alpha_global,
            w_rel=w_rel,
            w_cal=w_cal,
            w_tail=w_tail
        )
    return topk_ecr

K_eval = 10

# Baseline BPR 直接 top-K（不重排）
topk_bpr = get_topk_for_all_users(bpr_model, K=50)
metrics_bpr = compute_genre_metrics(topk_bpr, K_eval=K_eval)
print("==== Baseline BPR ====")
for k, v in metrics_bpr.items():
    print(f"{k}: {v:.4f}")

# 我们的 ECR
topk_ecr = get_topk_ecr_for_all_users(
    cand_items_dict,
    cand_scores_dict,
    K=K_eval,
    alpha_global=0.3,   # 0.2~0.4 之间可以试
    w_rel=0.5,
    w_cal=0.3,
    w_tail=0.2
)
metrics_ecr = compute_genre_metrics(topk_ecr, K_eval=K_eval)
print("\n==== ECR (Echo-Calibrated Re-ranking) ====")
for k, v in metrics_ecr.items():
    print(f"{k}: {v:.4f}")


==== Baseline BPR ====
NDCG@10: 0.0645
Recall@10: 0.0414
ILS@10: 0.2915
TailRatio@10: 0.0000
ECS1_hist_minus_rec_entropy: 0.1417
ECS2_KL_rec_hist: 1.3075

==== ECR (Echo-Calibrated Re-ranking) ====
NDCG@10: 0.0674
Recall@10: 0.0439
ILS@10: 0.2553
TailRatio@10: 0.0000
ECS1_hist_minus_rec_entropy: 0.0234
ECS2_KL_rec_hist: 0.5918
